In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

Load modeling parameter sweeps

In [ ]:
output_path = rd.rootdir/'output'/'fig_model-supp'
simulation_path = rd.rootdir/'output'/'modeling'/'julia_param_sweeps'/'per_param'/'sweep_df.gzip'
sim_data = pd.DataFrame()
if simulation_path.is_file(): 
    sim_data = pd.read_parquet(simulation_path)

In [ ]:
# Normalize parameter values such that original (middle) value = 1
def normalize_param_val(df):
    d = df.copy()
    vals = d['param_val'].unique()
    d['param_val_norm'] = d['param_val'] / (sorted(vals)[int(len(vals)/2)])
    return d

sim_data = sim_data.groupby('param')[sim_data.columns].apply(normalize_param_val).reset_index(drop=True)
display(sim_data)

In [ ]:
display(sim_data['param'].unique())

### Set up figure

In [ ]:
base_size = base.font_sizes['base_size']
smaller_size = base.font_sizes['smaller_size']

sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc={'font.size': base_size, 'font.family': 'sans-serif', 'font.sans-serif':['Arial']})
plt.rcParams.update({'axes.titlesize': base_size, 'axes.labelsize': base_size, 'xtick.labelsize': smaller_size, 'ytick.labelsize': smaller_size,
                     'pdf.fonttype': 42, 
                     'ytick.major.size': 3, 'xtick.major.size': 3, 'ytick.minor.size': 2, 'ytick.major.pad': 2, 'xtick.major.pad': 2, 
                     'lines.linewidth': 1,
                     'axes.spines.right': False, 'axes.spines.top': False, 'axes.labelpad': 2})

### Set up figure

In [ ]:
base_size = base.font_sizes['base_size']
smaller_size = base.font_sizes['smaller_size']

sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc={'font.size': base_size, 'font.family': 'sans-serif', 'font.sans-serif':['Arial']})
plt.rcParams.update({'axes.titlesize': base_size, 'axes.labelsize': base_size, 'xtick.labelsize': smaller_size, 'ytick.labelsize': smaller_size,
                     'pdf.fonttype': 42, 
                     'ytick.major.size': 3, 'xtick.major.size': 3, 'ytick.minor.size': 2, 'ytick.major.pad': 2, 'xtick.major.pad': 2, 
                     'lines.linewidth': 1,
                     'axes.spines.right': False, 'axes.spines.top': False, 'axes.labelpad': 2})

In [ ]:
# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(6.8504,7.625))
fig_gridspec = matplotlib.gridspec.GridSpec(5, 3, figure=fig,
    wspace=0.4, hspace=0.4, height_ratios=[1.5]*3+[1.625,1.5], width_ratios=[2.5,(6.8504-2.5)/2,(6.8504-2.5)/2])
subfigures = {
    'A': fig.add_subfigure(fig_gridspec[:3,0]),
    'B': fig.add_subfigure(fig_gridspec[0,1:]),
    'C': fig.add_subfigure(fig_gridspec[1,1:]),
    'D': fig.add_subfigure(fig_gridspec[2,1:]),
    'E': fig.add_subfigure(fig_gridspec[3,:2]),
    'F': fig.add_subfigure(fig_gridspec[3,2]),
    'G': fig.add_subfigure(fig_gridspec[4,:2]),
    '': fig.add_subfigure(fig_gridspec[4,2]),
}
for label, subfig in subfigures.items():
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))
scatter_kwargs = dict(s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
xlim = (2e2,6e4)
ylim = (2e1,1e5)

fig_path = output_path/'fig_model.pdf'
fig.savefig(rd.outfile(fig_path))

In [ ]:
# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(6.8504,3.25))
fig_gridspec = matplotlib.gridspec.GridSpec(2, 3, figure=fig)
subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:]),
    'B': fig.add_subfigure(fig_gridspec[1,0]),
    'C': fig.add_subfigure(fig_gridspec[1,1]),
    '': fig.add_subfigure(fig_gridspec[1,2]),
}
for label, subfig in subfigures.items():
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

fig_path = output_path/'fig_model-supp.pdf'
fig.savefig(rd.outfile(fig_path))

greys = matplotlib.colors.ListedColormap(matplotlib.colormaps['Greys'](np.linspace(0.2,0.7,256)))

In [ ]:
subfig = subfigures['A']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.7, bottom=0.4, top=0.5, right=0.6)
axes = subfig.subplots(1,5, gridspec_kw=dict(wspace=0.25, hspace=0.5))

param_labels = {'r_splicing': 'Splicing, $r_{splicing}$', 'δ_im': 'Primary transcript\ndeg., $δ_{immature}$', 'r_drosha': 'Dicer processing,\n$r_{drosha}$',
                'δ_mi': 'Free microRNA deg.,\n$δ_{mi}$', 'k_miRNA_bind': 'RISC loading,\n$k_{miRNA,bind}$',}
last = list(param_labels.keys())[-1]

# model: param sweeps
for param, ax in zip(param_labels.keys(), axes):
    yticks = np.linspace(0,1e4,5)
    plot_df = sim_data[sim_data['param']==param]
    sns.lineplot(data=plot_df, x='copy_num', y='protein', hue='param_val_norm', palette=greys, 
                ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend=('full' if param==last else False))
    ax.set(xticks=np.linspace(0,100,5), yticks=yticks, xlabel='', ylim=(0,12e3), ylabel='',
           yticklabels=['']*len(yticks),)
    ax.set_title(param_labels[param], color=base.colors['purple'])
    
axes[0].set(yticklabels=[f'{l:,.0f}' for l in yticks], ylabel='protein',)
axes[2].set(xlabel='copy #')

# adjust legend
ax = axes[-1]
leg = [f'{float(l.get_text()):.1f}x' for l in ax.legend().get_texts()]
sns.move_legend(ax,  loc='upper left', bbox_to_anchor=(1,1.2), frameon=False, labels=leg,
                fontsize=smaller_size, handlelength=1, labelspacing=0.3)
    
fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.7, bottom=0.4, top=0.5, right=0.65)
ax = subfig.subplots(1,1)

param_labels = {'k_deg': 'microRNA-mediated\nmRNA deg., $k_{deg}$',}
yticks = np.linspace(0,1e4,5)
param = list(param_labels.keys())[0]
plot_df = sim_data[sim_data['param']==param]
sns.lineplot(data=plot_df, x='copy_num', y='protein', hue='param_val_norm', palette=greys, 
            ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend='full')
ax.set(xticks=np.linspace(0,100,5), yticks=yticks, ylim=(0,12e3), xlabel='copy #',
       yticklabels=[f'{l:,.0f}' for l in yticks])
ax.set_title(param_labels[param], color=base.colors['red'])

# adjust legend
leg = [f'{float(l.get_text()):.1f}x' for l in ax.legend().get_texts()]
sns.move_legend(ax,  loc='upper left', bbox_to_anchor=(1,1.2), frameon=False, labels=leg,
                fontsize=smaller_size, handlelength=1, labelspacing=0.3)

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.7, bottom=0.4, top=0.5, right=0.65)
ax = subfig.subplots(1,1)

param_labels = {'δ_m': 'Free mRNA deg.,\n$δ_{m}$',}
yticks = np.linspace(0,1e4,5)
param = list(param_labels.keys())[0]
plot_df = sim_data[sim_data['param']==param]
sns.lineplot(data=plot_df, x='copy_num', y='protein', hue='param_val_norm', palette=greys, 
            ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend='full')
ax.set(xticks=np.linspace(0,100,5), yticks=yticks, ylim=(0,12e3), xlabel='copy #',
       yticklabels=[f'{l:,.0f}' for l in yticks], title=param_labels[param])

# adjust legend
leg = [f'{float(l.get_text()):.1f}x' for l in ax.legend().get_texts()]
sns.move_legend(ax,  loc='upper left', bbox_to_anchor=(1,1.2), frameon=False, labels=leg,
                fontsize=smaller_size, handlelength=1, labelspacing=0.3)

fig.savefig(rd.outfile(fig_path))